# CS 412 Machine Learning 2020 

# Assignment 3

100 pts

## Goal 

The goal of this assignment 

*  Introduction to working with text data
*  Gain experience with the Scikit-Learn library
*  Gain experience with Naive Bayes and Logistic Regression

## Dataset

**20 Newsgroup Dataset** is a collection 18846 documents which are about 20 different topics.


## Task
Build naive bayes and logistic regression classifiers with the scikit-learn library function to **classify** the documents about their content topic.

## Submission

Follow the instructions at the end.

# 1) Initialize

First, make a copy of this notebook in your drive

# 2) Load Dataset

The 20 Newsgroup Dataset exist on Scikit-Learn library.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
train_batch = fetch_20newsgroups(subset='train')
test_batch = fetch_20newsgroups(subset='test')

In [ ]:
# target groups you will be dealing with
target_groups = train_batch.target_names
target_groups

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
# creating training and test sets
train_x =  train_batch["data"]
train_y =  train_batch["target"]
test_x  =  test_batch["data"]
test_y  =  test_batch["target"]

In [ ]:
print(train_x[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [ ]:
print(target_groups[train_y[0]])

rec.autos


# Preprocess

In [ ]:
import re

In [ ]:
%%capture
import nltk
nltk.download("stopwords")

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
# You will use this function to preprocess your data. If you would like to add another preprocessing step in the function, please add it and mention about it in your report.
def preprocess(text):
    text = re.sub("[\w\d._]+@[^\s]+|[^\s]+\.[^\s]+|[^\s]+-[^\s]+|\d+|[^\w\s]","",text.lower().strip())
    text = ' '.join([stemmer.stem(word) for word in re.findall("\w+",text) if word not in stop_words])
    return text

In [ ]:
# Apply <preprocess> function on the training and test set 
trainx=[]
testx=[]
for txt in train_x:
  trainx.append(preprocess(txt))
for t in test_x:
  testx.append(preprocess(t))

#([word.lower() for word in txt.split()] for txt in text)

# Models

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

## Tune Naive Bayes

In [ ]:
# Create a CountVectorizer for NB with:
#     min_df = 50
#     max_df = 3000
#     stop_words = stop_words
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Vectorize your training and test set
vectorizer = CountVectorizer(min_df=50,max_df=3000,stop_words=stop_words)
train_x = vectorizer.fit_transform(trainx)
test_x = vectorizer.transform(testx)

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

#Initiate the NB model with required components.
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
model = MultinomialNB()

#model = model.fit(train_x,train_y)
#print(predictions)
#accuracy_score(test_y,predictions)

from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

hyperparameters=dict(alpha = (0.1,0.5,1.0,5.0))
mnb_grid_search = GridSearchCV(model,hyperparameters,cv=3,scoring="accuracy")
#Set the hyperparameter space that will be scanned:
#   alpha = (0.1,0.5,1.0,5.0)

mnb_grid_search.fit(train_x,train_y)
#Let the GridSearchCV scan the hyperparameter and find the best hyperparameter set that will maximize the scoring option.
#   cv = 3
#   scoring = "accuracy"


GridSearchCV(cv=3, error_score=nan,
             estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                     fit_prior=True),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': (0.1, 0.5, 1.0, 5.0)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
# show the best score
mnb_grid_search.best_score_


0.8212835269890522

In [ ]:
# show the best parameter
mnb_grid_search.best_params_

{'alpha': 0.5}

### Evaluate The Best Model for NB

In [ ]:
#Create your NB model with the best parameter set.
modelnb=MultinomialNB().set_params(**mnb_grid_search.best_params_)
#Fit your model on training set.
modelnb.fit(train_x,train_y)

MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True)

In [ ]:
# Make predictions on test set
predictions = modelnb.predict(test_x)


In [ ]:
# Show your accuracy on test set
accuracy_score(test_y,predictions)


0.7555762081784386

## Tune Logistic Regresion

In [ ]:
# Create a CountVectorizer for LR with:
#     min_df = 50
#     max_df = 3000
#     stop_words = stop_words

vectorizer = CountVectorizer(min_df=50,max_df=3000,stop_words=stop_words)

In [ ]:
train_x=trainx
test_x=testx

In [ ]:
# Vectorizer your training and test set
train_x = vectorizer.fit_transform(train_x)
test_x = vectorizer.transform(test_x)

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression
#Initiate the LR model:
#   max_iter=2000
model = LogisticRegression(max_iter=2000)
# Set the hyperparameter space that will be scanned:
hyperparameters=dict(C = (0.001,0.01,0.1,1))

#Let the GridSearchCV scan the hyperparameter and find the best hyperparameter set that will maximize the scoring option.
mnb_grid_search = GridSearchCV(model,hyperparameters,cv=3,scoring="accuracy")
mnb_grid_search.fit(train_x,train_y)


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=2000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': (0.001, 0.01, 0.1, 1)}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring='accuracy',
             verbose=0)

In [ ]:
# show the best score
mnb_grid_search.best_score_

0.8338344038554356

In [ ]:
# show the best parameter
mnb_grid_search.best_params_

{'C': 0.1}

### Evaluate The Best Model for Logistic Regression

In [ ]:
#Create your LR model with the best parameter set.

#Fit your model on training set.
#Create your NB model with the best parameter set.
mnb = LogisticRegression().set_params(**mnb_grid_search.best_params_)
#Fit your model on training set.
model = mnb.fit(train_x,train_y)



In [ ]:
# Make predictions on test set
predictions = model.predict(test_x)

In [ ]:
# Show your accuracy on test set
accuracy_score(test_y,predictions)

0.7484067976633032

# Feature Importances

In [ ]:
# Find the each category's most important top 3 features (words) for LR model and show with a dataframe

model.intercept_

array([-0.54960625,  0.45205302,  0.17583633,  0.11541413,  0.15562647,
        0.39304778,  0.28282377,  0.36353271,  0.19521149,  0.52677985,
        0.08893367, -0.88296326,  0.18424785,  0.09863297, -0.08303591,
        0.09424644, -0.47187842, -0.08783499, -0.58253194, -0.46853571])

In [ ]:
len(model.coef_.flatten())

61980

In [ ]:
model.coef_

array([[-0.08145479,  0.00948803,  0.00357308, ..., -0.0326397 ,
        -0.03607288, -0.02368216],
       [ 0.05160398,  0.04540452,  0.09167477, ...,  0.12149052,
        -0.02955457, -0.02209316],
       [ 0.07426539,  0.30845706, -0.1637374 , ...,  0.08232542,
         0.03557833, -0.01945549],
       ...,
       [ 0.07450212, -0.03540019,  0.00247345, ..., -0.02389627,
         0.00102889,  0.05903299],
       [-0.12065637, -0.09144288, -0.03498675, ..., -0.03530856,
         0.02293629, -0.05412505],
       [-0.09281603, -0.0196582 , -0.07123062, ..., -0.02851327,
        -0.00591538, -0.00191425]])

In [ ]:
coe = model.coef_
dict_list=[]

for feature in model.coef_:
  words = vectorizer.get_feature_names()
  cs=[str(a) for a in feature]
  coef_word = dict(sorted(list(zip(cs,words)),reverse=True,key=lambda x: x[0]))
  dict_list.append(coef_word)

In [ ]:
lr_df=pd.DataFrame(coe).transpose()
lr_df.columns=target_groups

for c in lr_df.columns:
  lr_df[c]=sorted(lr_df[c],reverse=True)
lr_df[:3]

,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
0,0.893657,1.175977,1.263341,0.708757,1.408501,0.991748,1.300283,1.480843,1.921220,1.134501,1.166319,1.072682,0.977359,0.825510,1.495623,0.905261,1.085459,1.171696,0.725268,0.825966
1,0.870171,0.939855,0.656691,0.577758,1.131229,0.958038,0.872831,0.734232,1.516526,0.761408,0.900092,1.003136,0.747706,0.694824,1.207823,0.684404,0.875022,1.130052,0.614036,0.697286
2,0.691037,0.589354,0.648192,0.543168,1.002968,0.898643,0.839952,0.653770,1.129837,0.684768,0.865401,0.730363,0.691502,0.691101,0.888761,0.681559,0.764017,0.694644,0.539981,0.612675


In [ ]:
for x in lr_df.columns:
  lr_df[x] = lr_df[x].astype(str)

for c in range(0,len(lr_df.columns)):
  for i in range(0,3):
    coef=lr_df._get_value(i, lr_df.columns[c])
    lr_df.at[i, lr_df.columns[c]] = dict_list[c].get(coef)

coef_word

lr_df[:3]

,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
0,atheist,graphic,window,monitor,mac,xr,sale,car,dod,basebal,hockey,clipper,circuit,doctor,space,christian,gun,israel,theodor,christian
1,keith,imag,win,gateway,appl,widget,offer,automot,bike,philli,team,encrypt,electron,diseas,orbit,god,waco,isra,clinton,order
2,islam,anim,driver,motherboard,powerbook,motif,forsal,auto,motorcycl,yanke,playoff,tap,tv,medic,pat,church,firearm,serdar,libertarian,koresh


In [ ]:
# Find the each category's most important top 3 features (words) for NB model and show with a dataframe
co = modelnb.coef_
dic_list=[]

for feature in co:
  words = vectorizer.get_feature_names()
  coefs=[str(a) for a in feature]
  coef_word = dict(sorted(list(zip(coefs,words)),reverse=True,key=lambda x: x[0]))
  dic_list.append(coef_word)

In [ ]:
nb_df=pd.DataFrame(co).transpose()
nb_df.columns=target_groups

for col in nb_df.columns:
  nb_df[col]=sorted(nb_df[col],reverse=True)
nb_df[:3]

,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
0,-4.422438,-4.141202,-3.601376,-3.870892,-4.336611,-4.237783,-4.116655,-3.712061,-4.190736,-4.439585,-4.190412,-4.155218,-4.768406,-5.320275,-4.078717,-4.026771,-4.154680,-4.410863,-4.765390,-4.570874
1,-4.679518,-4.513675,-4.134643,-4.594651,-4.593196,-4.429398,-4.638480,-5.182257,-4.608416,-4.487552,-4.235605,-4.422415,-5.006477,-5.361551,-4.940102,-4.436170,-4.771248,-4.649685,-5.048942,-4.627170
2,-4.812925,-4.666765,-4.664562,-4.603117,-4.719787,-4.644046,-4.812541,-5.215107,-4.749800,-4.733939,-4.436031,-4.697043,-5.096840,-5.377891,-5.010017,-4.832169,-5.072002,-4.772704,-5.054778,-4.795730


In [ ]:
for t in nb_df.columns:
  nb_df[t] = nb_df[t].astype(str)

for c in range(0,len(nb_df.columns)):
  for i in range(0,3):
    coef=nb_df._get_value(i, nb_df.columns[c])
    nb_df.at[i, nb_df.columns[c]] = dic_list[c].get(coef)
nb_df[:3]

,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
0,god,imag,window,drive,mac,window,sale,car,bike,year,team,key,wire,peopl,space,god,gun,armenian,peopl,god
1,peopl,file,file,card,appl,file,new,engin,dod,game,game,encrypt,ground,msg,orbit,christian,peopl,peopl,go,christian
2,say,graphic,driver,scsi,problem,program,offer,good,ride,team,play,chip,work,medic,launch,peopl,right,israel,think,peopl


# **Write your report in this cell**

The problem is to classify the words in a text data using two models; Naive Bayess and Logistic Regression.
We did preprocessing in order to extract unnecessary words in data which are not capable of being classified as we want, aldo extraxting the punctuation ad have a data which is as clean as possible in order to get the best results in models.

In NB, the best alpha was 0.5 and we got classification accuracy of %75 on test data.
In LR, the best C parameter was 0.1 and we got accuracy score of %74 on test data.

The most important features which are used to classify the texts can be seen in the lr_df and nb_df dataframes. A feature being the most important means it has the highest coefficient among other features and it has the most impact on deciding which class the text will be in. 

